In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("/kaggle/input/caloriecast-adaptive-ensemble-engine-for-s5e5/submission.csv")
df1.to_csv( 'adaptive-ensemble-engine_submission.csv', index=False)  #0.05670

df2 = pd.read_csv("/kaggle/input/ensemble-of-solutions/submission.csv")
df2.to_csv( 'Ensemble_submission.csv', index=False)   # 0.05668

df3 = pd.read_csv("/kaggle/input/ps-s5e5-log-blended-cat-xgboost-with-50-fold-cv/ensemble_submission.csv")
df3.to_csv( 'log-blended-cat-xgboost-with-50-fold-cv_submission.csv', index=False)  # 0.05670

df4 = pd.read_csv("/kaggle/input/ps-s5e5-cross-masked-ensemble/submission.csv")
df4.to_csv( 'cross-masked-ensemble_submission.csv', index=False)  # 0.05680

df5 = pd.read_csv("/kaggle/input/ps-s5e5-calorie-prediction-ensemble/submission.csv")
df5.to_csv( 'trippleEnsemble_submission.csv', index=False)     # 0.05667 

df6 = pd.read_csv("/kaggle/input/ps-s5e5-catboost-xgboost-feature-engineering/submission.csv")
df6.to_csv( 'catboost-xgboost-feature-engineering_submission.csv', index=False)   # 0.05679

df7 = pd.read_csv("/kaggle/input/pce-v1/submission.csv")
df7.to_csv( 'XG_Catboost_FE_submission.csv', index=False)   # 0.05668

df8 = pd.read_csv("/kaggle/input/calorie-best-fe/submission.csv")
df8.to_csv( 'calorie-best-fe_submission.csv', index=False)   # 0.05667


#df9 = pd.read_csv("/kaggle/input/calories-expenditure/submission.csv")  # 0.05692
#df10 = pd.read_csv("/kaggle/input/best-fe-3-diverse-models/submission_.csv") #0.05669


df11 = pd.read_csv("/kaggle/input/calorie-expenditure-female-vs-male/submission.csv")
df11.to_csv( 'male_female_submission.csv', index=False)   # 0.05667


df12 = pd.read_csv("/kaggle/input/kmeans-clustering-based-fe-improves-cv-lb-scores/submission.csv")
df12.to_csv( 'kmean_submission.csv', index=False)   # 0.05695 V2


df13 = pd.read_csv("/kaggle/input/s5e5-xgb-cat-lgbm-meta-models/submission.csv")
df13.to_csv( 'metamodel_submission.csv', index=False)   # 0.05695 V2



ground_truth = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")  







In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

def generate_ensemble_submissions(model_dfs, base_submission, output_prefix="submission"):

    all_preds = np.stack([df['Calories'].values for df in model_dfs], axis=1)
    
    # Mean
    mean_preds = np.mean(all_preds, axis=1)
    base_submission['Calories'] = mean_preds
    base_submission.to_csv(f'{output_prefix}_mean.csv', index=False)
    
    # Median
    median_preds = np.median(all_preds, axis=1)
    base_submission['Calories'] = median_preds
    base_submission.to_csv(f'{output_prefix}_median.csv', index=False)

    # Trimmed mean (exclude top and bottom 2 predictions)
    trimmed_preds = np.mean(np.sort(all_preds, axis=1)[:, 2:-2], axis=1)
    base_submission['Calories'] = trimmed_preds
    base_submission.to_csv(f'{output_prefix}_trimmed_mean.csv', index=False)

    # Geometric mean
    geo_mean_preds = np.exp(np.mean(np.log(all_preds + 1e-8), axis=1))  # small epsilon to avoid log(0)
    base_submission['Calories'] = geo_mean_preds
    base_submission.to_csv(f'{output_prefix}_geo_mean.csv', index=False)

    # Rank averaging
    ranks = np.argsort(np.argsort(all_preds, axis=0), axis=0)
    avg_ranks = np.mean(ranks, axis=1)
    norm_ranks = avg_ranks / avg_ranks.max() * all_preds.mean()
    base_submission['Calories'] = norm_ranks
    base_submission.to_csv(f'{output_prefix}_rank_avg.csv', index=False)

    # Ridge regression meta-model (stacking)
    X = all_preds
    y = np.mean(all_preds, axis=1)  # fake target, just to blend
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_scaled, y)
    ridge_preds = ridge.predict(X_scaled)
    base_submission['Calories'] = ridge_preds
    base_submission.to_csv(f'{output_prefix}_ridge_stack.csv', index=False)

    print("Ensembles saved as CSVs.")

# Usage
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df11, df12, df13]
ground_truth = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")
generate_ensemble_submissions(dfs, ground_truth.copy())


Ensembles saved as CSVs.
